# Imports

In [145]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Read the default SQAD data

In [146]:
df = pd.read_csv("sqad_dataframe.csv")
df["question"] = df["question"].astype("string")
df["text"] = df["text"].astype("string")
df["answer"] = df["answer"].astype("string")
df["answer_sentence"] = df["answer_sentence"].astype("string")

In [147]:
df.rename(columns = {"Unnamed: 0" :"question_number"}, inplace = True)
df.dtypes

question_number     int64
question           string
text               string
answer             string
answer_sentence    string
dtype: object

# Preprocess data

## Split text into array of sentences

In [148]:
df["text_sentences"] = df.text.map(lambda x: x.split("\n"))
df["text"] = df["text"].str.replace('\n' , ' ')
df["question"] = df["question"].str.replace('\n' , ' ')
df["answer"] = df["answer"].str.replace('\n' , ' ')
df["answer_sentence"] = df["answer_sentence"].str.replace('\n' , ' ')
df.head()

,question_number,question,text,answer,answer_sentence,text_sentences
0,1,Kdo je autorem novely Létající jaguár?,Létající jaguár je novela spisovatele Josefa F...,Josefa Formánka,Létající jaguár je novela spisovatele Josefa F...,[Létající jaguár je novela spisovatele Josefa ...
1,2,Jak se nazývá věda zabývající se houbami?,"Houby (Fungi, dříve Mycophyta) představují vel...",mykologie,Věda zabývající se houbami se nazývá mykologie.,"[Houby (Fungi, dříve Mycophyta) představují ve..."
2,3,"Jak se nazývá strom, jehož zrna jsou využívána...",Čokoláda je obvyklá součást nejrůznějších druh...,Theobroma cacao,"Čokoláda se vyrábí z kvašených, pražených a ml...",[Čokoláda je obvyklá součást nejrůznějších dru...
3,4,Kdo se stal prvním prezidentem České republiky?,Václav Havel (5. října 1936 Praha – 18. prosin...,Václav Havel,Václav Havel (5. října 1936 Praha – 18. prosin...,[Václav Havel (5. října 1936 Praha – 18. prosi...
4,5,Do jaké čeledi rostlin patří pampeliška?,"Pampeliška (Taraxacum), či také smetánka, je z...",hvězdnicovité,"Pampeliška (Taraxacum), či také smetánka, je z...","[Pampeliška (Taraxacum), či také smetánka, je ..."


### Random exaple 

In [156]:
print("QUESTION: " + df["question"].iloc[123])
print("\nANSWER: " + df["answer"].iloc[123])
print("\nTEXT: " + df["text"].iloc[123][:5000])
print("\nANSWER SENTENCE: " + df["answer_sentence"].iloc[123])


QUESTION: Kterého dne zemřel Alfons Mucha?

ANSWER: 14. července 1939

TEXT: Alfons Maria Mucha (24. července 1860 Ivančice - 14. července 1939 Praha) byl český malíř a designér období secese, které je v angličtině a francouzštině známo pod francouzským označením Art nouveau (nové umění). Alfons Mucha byl otcem spisovatele Jiřího Muchy. Stal se známým téměř přes noc díky divadelnímu plakátu Gismonda (1894-1895), který si u něj objednala pařížská herečka Sarah Bernhardtová. Herečka později uzavřela s Muchou smlouvu a on pro ni a Divadlo Renesance v Paříži vytvořil ještě několik dalších plakátů. Jeho životním dílem je cyklus velkoformátových pláten Slovanská epopej, na kterém pracoval osmnáct let. Krom toho se také zasloužil spolu s Jaroslavem Kvapilem a Ladislavem Syllabou o obnovení svobodného zednářství v Československu. Narodil se v Ivančicích v rodině soudního zřízence Ondřeje Muchy a jeho manželky Amálie rozené Malé. Měl dvě vlastní sestry, Annu a Andělu. Studoval na Slovanském gym

## Replace YES/NO type answers with whole answer sentence

In [157]:
df_wo_yes_no_answers = df.copy()

df_wo_yes_no_answers["answer"] = np.where(df_wo_yes_no_answers["answer"].str.lower().str.contains("ano|ne"),
                                          df_wo_yes_no_answers["answer_sentence"], 
                                          df_wo_yes_no_answers["answer"])

df_wo_yes_no_answers[df_wo_yes_no_answers["answer"].isin(["ano", "ne"])].answer.count()

0

## Checking if some texts do not contain the answer sentence

In [158]:
df_wo_yes_no_answers[["answer", "answer_sentence"]]
bad_row = df_wo_yes_no_answers.iloc[0]
try:
    for row in df_wo_yes_no_answers.iloc:
        if row["answer_sentence"] not in row["text_sentences"]:
            bad_row = row
            raise ValueError()
except ValueError:
    print(row["answer_sentence"])
else:
    print("It's fine")

It's fine


## Get index of answer_sentence from text_sentences

In [159]:
def get_sentence_index(row):
    sentences = row["text_sentences"]
    answer_sentence = row["answer_sentence"]
    return sentences.index(answer_sentence)

#### Test

In [160]:
print(get_sentence_index(df_wo_yes_no_answers.iloc[8]))
print(df_wo_yes_no_answers.iloc[8]["answer_sentence"])
for i, s in enumerate(df_wo_yes_no_answers.iloc[8]["text_sentences"]):
    print(i," >>> ",s)

0
Nespoutaný Django (v anglickém originále Django Unchained) je americký western režiséra Quentina Tarantina z roku 2012.
0  >>>  Nespoutaný Django (v anglickém originále Django Unchained) je americký western režiséra Quentina Tarantina z roku 2012.
1  >>>  Film se odehrává v jižních státech USA a zaměřuje se na Djanga, osvobozeného otroka, který cestuje napříč Spojenými státy v doprovodu doktora Kinga Schultze, nájemného lovce lidí.
2  >>>  V hlavních rolích hrají Jamie Foxx, Christoph Waltz, Leonardo DiCaprio, Kerry Washingtonová a Samuel L. Jackson.
3  >>>  Snímek získal dva Zlaté glóby v kategoriích Nejlepší herec ve vedlejší roli (Christoph Waltz) a Nejlepší scénář (Quentin Tarantino), kromě toho byl ještě nominován v kategoriích Nejlepší film (drama), Nejlepší režie (Quentin Tarantino) a Nejlepší herec ve vedlejší roli (Leonardo DiCaprio).
4  >>>  Získal také dva Oscary v kategoriích Nejlepší herec ve vedlejší roli (Christoph Waltz) a Nejlepší původní scénář (Quentin Tarantino), 

In [161]:
x = 8
ind = get_sentence_index(df.iloc[x])
print(ind)
print(df["question_number"].iloc[x])
#df["question"].iloc[x] = 
print(df["answer_sentence"].iloc[x])
#df["answer_sentence"].iloc[x] = "Man-in-the-middle (zkratka MITM, z angličtiny \"člověk uprostřed\" nebo \"člověk mezi\") je v informatice název útoku na kryptografii."
print(df["answer"].iloc[x])
#df["answer"].iloc[x] = "pět"
print(df["answer"].iloc[x] in df["answer_sentence"].iloc[x])
print(df["text_sentences"].iloc[x][ind])
#df["text_sentences"].iloc[x][ind] = "Film produkoval Terry Gilliam a George Harisson."
#df["text"].iloc[x] = df["text"].iloc[x].replace("Film produkoval Terry Giliam a George Harisson.", "Film produkoval Terry Gilliam a George Harisson.")
print(df["text"].iloc[x])
#df.drop(2102, inplace="True")

0
9
Nespoutaný Django (v anglickém originále Django Unchained) je americký western režiséra Quentina Tarantina z roku 2012.
Quentina Tarantina
True
Nespoutaný Django (v anglickém originále Django Unchained) je americký western režiséra Quentina Tarantina z roku 2012.
Nespoutaný Django (v anglickém originále Django Unchained) je americký western režiséra Quentina Tarantina z roku 2012. Film se odehrává v jižních státech USA a zaměřuje se na Djanga, osvobozeného otroka, který cestuje napříč Spojenými státy v doprovodu doktora Kinga Schultze, nájemného lovce lidí. V hlavních rolích hrají Jamie Foxx, Christoph Waltz, Leonardo DiCaprio, Kerry Washingtonová a Samuel L. Jackson. Snímek získal dva Zlaté glóby v kategoriích Nejlepší herec ve vedlejší roli (Christoph Waltz) a Nejlepší scénář (Quentin Tarantino), kromě toho byl ještě nominován v kategoriích Nejlepší film (drama), Nejlepší režie (Quentin Tarantino) a Nejlepší herec ve vedlejší roli (Leonardo DiCaprio). Získal také dva Oscary v kat

In [162]:
class MismatchError(Exception):
    pass

In [182]:
#tmp = 5631
#start_sentence_index = df_wo_yes_no_answers["text"].iloc[tmp].index(df_wo_yes_no_answers["answer_sentence"].iloc[tmp])
#print(start_sentence_index)
#print(df_wo_yes_no_answers["answer_sentence"].iloc[tmp])
#print(df_wo_yes_no_answers["text"].iloc[tmp][start_sentence_index:(start_sentence_index+len(df_wo_yes_no_answers["answer_sentence"].iloc[tmp]))])

data = []
for row in tqdm(df_wo_yes_no_answers.iloc, desc="Comparing found answer sentences with true answer sentences and creating SQuAD-format dataframe", total=13473):
    try:
        start_sentence_index = row.text.index(row.answer_sentence)
        answer_sentence = row.answer_sentence
        found_answer_sentence = row.text[start_sentence_index:(start_sentence_index+len(row.answer_sentence))]
        exact_answer = None
        #row.answer.split('#')[0].strip()
        answers = row.answer.split('#')
        for ans in  row.answer.split('#'):
            if ans in answer_sentence:
                exact_answer = ans.strip()
                break
        if exact_answer is None:
            continue
        answer_start = start_sentence_index + answer_sentence.lower().index(exact_answer.lower())
        found_answer = row.text[answer_start:(answer_start)+len(exact_answer)]
        #print(row.question)
        #print(repr(exact_answer))
        #print(repr(found_answer))
        if found_answer.lower() != exact_answer.lower():
                continue
        data.append([row.question_number, row.question, row.text, answer_sentence, exact_answer, answer_start])
    except ValueError:
        print("Answer sentence start: ", start_sentence_index)
        print("Answer sentence  text: ", answer_sentence)
        print("Answer start index: ", answer_start)
        #print("added to start index: ",answer_sentence.lower().index(exact_answer.lower()))
        print("Found answer sentence: ", found_answer_sentence)
        print("Found answer: ", found_answer)
        print("Exact answer: ",exact_answer)
        print(row.question_number)
print()
print("Task completed")
squad_format = pd.DataFrame(data, columns=["question_number", "question", "text", "answer_sentence", "answer", "answer_start"])
squad_format.head()

Comparing found answer sentences with true answer sentences and creating SQuAD-format dataframe: 100%|█| 13473/13473 [00


All indexed answer sentences are correct


,question_number,question,text,answer_sentence,answer,answer_start
0,1,Kdo je autorem novely Létající jaguár?,Létající jaguár je novela spisovatele Josefa F...,Létající jaguár je novela spisovatele Josefa F...,Josefa Formánka,38
1,2,Jak se nazývá věda zabývající se houbami?,"Houby (Fungi, dříve Mycophyta) představují vel...",Věda zabývající se houbami se nazývá mykologie.,mykologie,798
2,3,"Jak se nazývá strom, jehož zrna jsou využívána...",Čokoláda je obvyklá součást nejrůznějších druh...,"Čokoláda se vyrábí z kvašených, pražených a ml...",Theobroma cacao,287
3,4,Kdo se stal prvním prezidentem České republiky?,Václav Havel (5. října 1936 Praha – 18. prosin...,Václav Havel (5. října 1936 Praha – 18. prosin...,Václav Havel,0
4,5,Do jaké čeledi rostlin patří pampeliška?,"Pampeliška (Taraxacum), či také smetánka, je z...","Pampeliška (Taraxacum), či také smetánka, je z...",hvězdnicovité,86


### Check if the answer_start and answer columns are set correctly

In [200]:
try:
    for index, row in enumerate(squad_format.iloc):
        found_answer = row.text[row.answer_start:row.answer_start+len(row.answer)]
        #print(row.answer, found_answer, row.answer.lower() == found_answer.lower())
        if row.answer.lower() != found_answer.lower():
            print(row.answer, found_answer)
            raise ValueError
except ValueError:
        print("Some rows are not correct")
else:
    print("Done, all rows are correct")
    

Done, all rows are correct


In [201]:
squad_format

,question_number,question,text,answer_sentence,answer,answer_start
0,1,Kdo je autorem novely Létající jaguár?,Létající jaguár je novela spisovatele Josefa F...,Létající jaguár je novela spisovatele Josefa F...,Josefa Formánka,38
1,2,Jak se nazývá věda zabývající se houbami?,"Houby (Fungi, dříve Mycophyta) představují vel...",Věda zabývající se houbami se nazývá mykologie.,mykologie,798
2,3,"Jak se nazývá strom, jehož zrna jsou využívána...",Čokoláda je obvyklá součást nejrůznějších druh...,"Čokoláda se vyrábí z kvašených, pražených a ml...",Theobroma cacao,287
3,4,Kdo se stal prvním prezidentem České republiky?,Václav Havel (5. října 1936 Praha – 18. prosin...,Václav Havel (5. října 1936 Praha – 18. prosin...,Václav Havel,0
4,5,Do jaké čeledi rostlin patří pampeliška?,"Pampeliška (Taraxacum), či také smetánka, je z...","Pampeliška (Taraxacum), či také smetánka, je z...",hvězdnicovité,86
...,...,...,...,...,...,...
12438,13468,Kterou společností bylo zakoupeno 6 vozů k dop...,"Metro v Norimberku, německy Nürnberg U-Bahn, j...",V roce 2004 bylo společností VGA zakoupeno 6 v...,VGA,1191
12439,13469,Mají borelovské množiny název po Émile Borelovi?,Borelovská množina v matematice je libovolná m...,Název mají po francouzském matematikovi Émile ...,Název mají po francouzském matematikovi Émile ...,236
12440,13471,Je Thomas Hylland Eriksen neznámý norský spiso...,"Thomas Hylland Eriksen (* 6. února 1962, Oslo,...","Thomas Hylland Eriksen (* 6. února 1962, Oslo,...","Thomas Hylland Eriksen (* 6. února 1962, Oslo,...",0
12441,13472,Kterou technologií vysílá televizní kanál Trojka?,Trojka byla třetí slovenský kanál Slovenské te...,Televizní kanál vysílá technologií DVB-T a tak...,DVB-T,323


# Convert to JSON 

In [206]:
import json

In [210]:
"""
{
    "answers": {
        "answer_start": [1],
        "text": ["This is a test text"]
    },
    "context": "This is a test context.",
    "id": "1",
    "question": "Is this a test?",
    "title": "train test"
}
"""

json_data = []
for row in tqdm(squad_format.iloc, desc="Creating JSON objects", total=12443):
    data = {
                "answers": {
                  "answer_start": [int(row.answer_start)],
                  "text": [row.answer]
                },
                "context": row.text,
                "id": int(row.question_number),
                "question": row.question,
                "title": row.question
            }
    json_data.append(data)

json_string = json.dumps(json_data)
with open("sqad.json", "w") as json_file:
    json_file.write(json_string)

creating JSON objects: 100%|████████████████████████████████████████████████████| 12443/12443 [00:01<00:00, 9198.28it/s]


To be continued...